# Session 6: AI API 호출(Inference) 및 프로젝트 완성

### ■ Session Objective
-   Session 5에서 배포한 `loan-approval-service` Endpoint에 **Vertex AI SDK**를 사용하여 실시간 예측 요청을 전송합니다.
-   가상의 대출 신청자 데이터를 API가 요구하는 형식(JSON)으b로 변환하고, API의 응답(승인/거절)을 확인하여 1차시 프로젝트를 완성합니다.
-   실습 완료 후, **엔드포인트 배포를 취소(Undeploy)하여 불필요한 비용 발생을 중단**합니다.

---

### ■ Key Concepts

1.  **Vertex AI SDK (Software Development Kit)**
    -   Python 코드에서 GCP 서비스(Vertex AI, GCS 등)와 상호작용할 수 있도록 Google이 제공하는 라이브러리입니다.
    -   복잡한 HTTP 요청 대신, `endpoint.predict()`와 같은 간단한 함수로 API를 호출할 수 있게 해줍니다.

2.  **API 요청 페이로드 (Request Payload)**
    -   API에 전송하는 실제 데이터 묶음입니다. Vertex AI Endpoint는 특정 **JSON 형식**을 요구합니다.
    -   **`{"instances": [ ... ]}`** 형태여야 합니다.

3.  **데이터 일관성 (Data Consistency)** - (⭐ **매우 중요** ⭐)
    -   API로 전송하는 데이터는 **Session 3에서 모델을 학습시킬 때 사용한 데이터와 100% 동일한 형태(피처 순서, 인코딩 방식)**여야 합니다.
    -   이 일관성을 맞추기 위해, Session 4에서 GCS에 함께 업로드했던 **`model_features.pkl`** 파일을 다시 불러와 사용할 것입니다.

---

### 🚨 (필수) 비용 관리: 실습 후 엔드포인트 정리
-   본 세션이 끝나면, **비용이 계속 청구되는 엔드포인트를 즉시 '배포 취소'해야 합니다.**
-   이 노트북의 **마지막 단계**에 배포 취소 가이드가 포함되어 있습니다.

In [ ]:
# ==============================================================================
# [실습 1] API 호출을 위한 환경 설정
# ==============================================================================
import pandas as pd
import joblib
from google.cloud import aiplatform  # Vertex AI SDK
from google.cloud import storage      # GCS SDK (피처 목록 로드용)

# --- (1/4) GCP 환경 변수 설정 ---
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ 수강생 각자의 GCP 프로젝트 ID로 이 부분을 수정해야 합니다! ※
# ※ GCP 콘솔 상단에서 프로젝트 ID를 복사해오세요.            ※
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
PROJECT_ID = "여러분의-GCP-PROJECT-ID"  # 예: "gcp-project-12345"
REGION = "asia-northeast3"             # 배포 리전

# --- (2/4) GCS 클라이언트 초기화 ---
# (Session 4에서 업로드한 'model_features.pkl'을 다운로드하기 위함)
gcs_client = storage.Client(project=PROJECT_ID)

# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ Session 4에서 사용한 버킷 이름과 피처 파일 경로를 정확히 입력합니다.
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
BUCKET_NAME = "여러분의-GCS-버킷-이름" # 예: "gcp-project-12345-loan-data"
FEATURES_GCS_PATH = "loan_model/artifacts/model_features.pkl" # Session 4에서 지정한 경로

# GCS에서 피처 목록 파일 로드
bucket = gcs_client.bucket(BUCKET_NAME)
blob = bucket.blob(FEATURES_GCS_PATH)
blob.download_to_filename("model_features.pkl") # Workbench 로컬에 임시 다운로드

# 로드된 피처 목록 확인
MODEL_FEATURES = joblib.load("model_features.pkl")
print(f"▶ GCS에서 {len(MODEL_FEATURES)}개의 피처 목록을 성공적으로 로드했습니다.")
# print(MODEL_FEATURES) # (확인용)

# --- (3/4) Vertex AI SDK 초기화 ---
aiplatform.init(project=PROJECT_ID, location=REGION)

# --- (4/4) 엔드포인트 객체 가져오기 ---
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ GCP 콘솔 'Vertex AI > 엔드포인트' 메뉴에서 
# ※ '엔드포인트 ID' (긴 숫자)를 복사하여 붙여넣어야 합니다.
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
ENDPOINT_ID = "여러분의-엔드포인트-ID" # 예: "1234567890123456789"

endpoint = aiplatform.Endpoint(endpoint_name=ENDPOINT_ID)
print(f"▶ 엔드포인트(ID: {ENDPOINT_ID})에 성공적으로 연결되었습니다.")

In [ ]:
# ==============================================================================
# [실습 2] 예측 요청 전송 및 결과 확인
# ==============================================================================

# 1. 가상의 대출 신청자 데이터 (Raw Data)
#    (Session 3의 시뮬레이션 데이터와 동일)
sample_data = {
    'Gender': 'Male', 'Married': 'Yes', 'Dependents': '1', 'Education': 'Graduate',
    'Self_Employed': 'No', 'ApplicantIncome': 5000, 'CoapplicantIncome': 2000,
    'LoanAmount': 150, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0,
    'Property_Area': 'Urban'
}
sample_df = pd.DataFrame([sample_data])
print(" [1. 원본 요청 데이터 (Raw Data)]")
print(sample_df)

# 2. API 전송을 위한 데이터 전처리 (학습 때와 동일하게)
print("\n [2. API 전송용 데이터 전처리]")
sample_df_processed = pd.get_dummies(sample_df)

# 3. 피처 정렬 (Feature Alignment) - ⭐ 핵심 ⭐
#    GCS에서 로드한 'MODEL_FEATURES' 리스트를 기준으로 컬럼을 재정렬합니다.
final_sample_df = sample_df_processed.reindex(columns=MODEL_FEATURES, fill_value=0)
print("  - 학습 시점과 동일하게 피처 정렬 완료.")

# 4. Vertex AI API가 요구하는 JSON 페이로드 형식으로 변환
#    DataFrame -> list of dictionaries
instances = final_sample_df.to_dict(orient='records')
print(f"  - API 요청 JSON 형식으로 변환 완료: {instances}")

# 5. API 호출 (실시간 예측 요청)
print("\n [3. Vertex AI Endpoint에 예측 요청 전송...]")
# endpoint.predict() 함수는 'instances' 리스트를 자동으로 JSON으로 변환하여 전송합니다.
prediction_result = endpoint.predict(instances=instances)

print("\n [4. API 응답 결과 (Raw Response)]")
print(prediction_result)

# 6. 결과 해석
print("\n [5. 최종 예측 결과 해석]")
# 응답 형식: prediction_result.predictions[0]
final_prediction = prediction_result.predictions[0]

if final_prediction == 1:
    print(f"  ▶▶ 예측 결과: '승인 (Approved - Y)'")
else:
    print(f"  ▶▶ 예측 결과: '거절 (Refused - N)'")

print("\n==============================================================================")
print("▶▶▶ 1차시 프로젝트 API 구축 및 테스트가 성공적으로 완료되었습니다! ◀◀◀")
print("==============================================================================")

# [실습 3] (필수!) 엔드포인트 배포 취소 (비용 중단)

축하합니다! 여러분은 1차시의 모든 과정을 완료하고, 클라우드에 배포된 AI API를 호출하는 데 성공했습니다.

이제 **불필요한 비용이 계속 발생하는 것을 막기 위해** 배포된 엔드포인트 서비스를 즉시 중지(배포 취소)해야 합니다.

---

### ■ 배포 취소 절차 (GCP Console UI - 권장)

1.  GCP 콘솔에서 **'Vertex AI' > '엔드포인트(Endpoints)'** 메뉴로 이동합니다.
2.  우리가 생성한 **`loan-approval-service`** 엔드포인트의 이름(링크)을 클릭합니다.
3.  상세 정보 페이지 상단에서 **`배포된 모델 배포 취소`** (휴지통 아이콘 🗑️) 버튼을 클릭합니다.
4.  팝업창에서 배포 취소할 모델을 확인하고 `배포 취소` 버튼을 클릭합니다.
5.  **(확인)** 잠시 후, '배포된 모델' 섹션이 비어있게 되면 비용 청구가 중지된 것입니다. (엔드포인트 자체는 목록에 남아있어도 괜찮습니다.)

### ■ (선택) 엔드포인트 완전 삭제

-   배포 취소 후, 엔드포인트 목록에서 `loan-approval-service`를 체크하고 상단의 `삭제` 버튼을 누르면 목록에서도 완전히 사라집니다.

---
### ■ Session 6 및 1차시 교육 완료
수고하셨습니다. 이제 2차시 교육에서 이 API를 '전문가 Tool'로 활용할 준비가 모두 끝났습니다.